# Quantized Deployment and Throughput/Accuracy Testing
This notebook deploys the quantized model on an FPGA and performs throughput and accuracy testing.
If the desired deployment is on a PYNQ board, this notebook should be run from the PYNQ host.

In [1]:
from finn.core.datatype import DataType
from driver_base import FINNExampleOverlay
import numpy as np
import proc_for_infer as pfi

In [2]:
#set up model info
# dictionary describing the I/O of the FINN-generated accelerator
io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : DataType.UINT4,
    "odt" : DataType.UINT4,
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : (1, 14),
    "oshape_normal" : (1, 1),
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : (1, 1, 14),
    "oshape_folded" : (1, 1, 1),
    "ishape_packed" : (1, 1, 7),
    "oshape_packed" : (1, 1, 1)
}
platform = "zynq-iodma"
bitfile = "output_final/deploy/bitfile/finn-accel.bit"
input_file = "data/X_test.npy"
output_comparison = "data/Y_test.npy"
runtime_weights = "runtime_weights/"
batch_size = 1

In [3]:
#create accelerator
accel = FINNExampleOverlay(bitfile_name=bitfile, platform=platform, 
                          io_shape_dict=io_shape_dict, batch_size=batch_size,
                          runtime_weight_dir=runtime_weights)

### Throughput Test
Run a basic throughput test and print the results

In [4]:
res = accel.throughput_test()
print(res)

{'runtime[ms]': 0.2818107604980469, 'throughput[images/s]': 3548.4805414551606, 'DRAM_in_bandwidth[Mb/s]': 0.024839363790186125, 'DRAM_out_bandwidth[Mb/s]': 0.0035484805414551608, 'fclk[mhz]': 99.999, 'batch_size': 1, 'fold_input[ms]': 5.602836608886719e-05, 'pack_input[ms]': 0.007915258407592773, 'copy_input_data_to_device[ms]': 0.0011277198791503906, 'copy_output_data_from_device[ms]': 0.0002923011779785156, 'unpack_output[ms]': 0.0011167526245117188, 'unfold_output[ms]': 3.457069396972656e-05}


### Accuracy Test
Perform inference on testing dataset and compare to actual values.

In [8]:
inp = np.load(input_file)
exp_out = np.load(output_comparison)
proc_inp = pfi.preproc(inp)
infs = []
#infs = accel.execute(proc_inp)
for proc_input in proc_inp:
    proc_input = proc_input.reshape((1,14))
    out = accel.execute(proc_input)
    inf = pfi.postproc(out)
    #print(inf)
    infs.append(inf)
print(infs)

[array([[ 338.267]]), array([[ 473.5738]]), array([[ 236.7869]]), array([[ 304.4403]]), array([[ 405.9204]]), array([[ 405.9204]]), array([[ 405.9204]]), array([[ 372.0937]]), array([[ 439.7471]]), array([[ 304.4403]]), array([[ 439.7471]]), array([[ 439.7471]]), array([[ 338.267]]), array([[ 405.9204]]), array([[ 405.9204]]), array([[ 338.267]]), array([[ 439.7471]]), array([[ 439.7471]]), array([[ 439.7471]]), array([[ 405.9204]]), array([[ 473.5738]]), array([[ 405.9204]]), array([[ 507.4005]]), array([[ 439.7471]]), array([[ 304.4403]]), array([[ 507.4005]]), array([[ 236.7869]]), array([[ 507.4005]]), array([[ 304.4403]]), array([[ 405.9204]]), array([[ 236.7869]]), array([[ 507.4005]]), array([[ 405.9204]]), array([[ 304.4403]]), array([[ 372.0937]]), array([[ 405.9204]]), array([[ 372.0937]]), array([[ 338.267]]), array([[ 304.4403]]), array([[ 405.9204]]), array([[ 507.4005]]), array([[ 270.6136]]), array([[ 338.267]]), array([[ 405.9204]]), array([[ 304.4403]]), array([[ 236.7

In [10]:
npinfs = np.asarray(infs)
npinfs = npinfs.reshape((12752,1))
print(npinfs.shape)
print(exp_out.shape)

(12752, 1)
(12752, 1)


In [14]:
err = npinfs - exp_out
print(exp_out)
running_err = 0.0
for val in err:
    print(val)
    se = val**2
    running_err += se
mse = running_err / 12752
print(mse)
#print(err)

[[  0.        ]
 [ 89.24584198]
 [  0.        ]
 ..., 
 [  0.        ]
 [  0.        ]
 [  0.        ]]
[ 338.267]
[ 384.32795802]
[ 236.7869]
[ 283.90870096]
[ 405.9204]
[ 405.9204]
[ 405.9204]
[ 372.0937]
[ 439.7471]
[ 304.4403]
[ 432.79116881]
[ 439.7471]
[ 338.267]
[ 369.57515845]
[ 405.9204]
[ 312.1745985]
[ 439.7471]
[ 397.34197152]
[ 439.7471]
[ 405.9204]
[ 421.82962886]
[ 405.9204]
[ 374.91411084]
[ 402.53105844]
[ 266.95361329]
[ 458.16715115]
[ 182.05429883]
[ 460.09940213]
[ 304.4403]
[ 405.9204]
[ 236.7869]
[ 426.07532758]
[ 405.9204]
[ 304.4403]
[ 335.29650457]
[ 405.9204]
[ 369.29164989]
[ 338.267]
[ 263.51736451]
[ 386.70034537]
[ 445.35966748]
[ 270.6136]
[ 338.267]
[ 405.9204]
[ 304.4403]
[ 202.51829572]
[ 248.56134437]
[ 405.9204]
[ 236.7869]
[ 304.4403]
[ 270.6136]
[ 304.4403]
[ 403.95202172]
[ 405.9204]
[ 370.63101707]
[ 405.9204]
[ 418.34747783]
[ 372.0937]
[ 304.4403]
[ 259.90956626]
[ 417.70033753]
[ 436.41553965]
[ 382.91685424]
[ 439.7471]
[ 411.64874948]
[ 372

[ 269.55745317]
[ 439.7471]
[ 304.4403]
[ 439.7471]
[ 358.71789298]
[ 339.8251415]
[ 326.76168422]
[ 439.7471]
[ 453.77868167]
[ 304.4403]
[ 409.49747384]
[ 405.9204]
[ 338.267]
[ 318.00594077]
[ 308.68086795]
[ 438.72828168]
[ 286.23547174]
[ 439.7471]
[ 410.37620843]
[ 242.57062532]
[ 245.88245796]
[ 439.7471]
[ 405.9204]
[ 236.7869]
[ 304.4403]
[ 405.9204]
[ 304.4403]
[ 374.9826957]
[ 473.5738]
[ 405.9204]
[ 356.6106576]
[ 308.12465627]
[ 405.9204]
[ 304.4403]
[ 439.7471]
[ 419.33794659]
[ 372.0937]
[ 236.7869]
[ 286.57188035]
[ 441.90411082]
[ 405.9204]
[ 270.6136]
[ 236.7869]
[ 439.7471]
[ 390.71457759]
[ 405.9204]
[ 236.7869]
[ 278.86606408]
[ 435.61258954]
[ 304.4403]
[ 447.31181231]
[ 405.9204]
[ 439.7471]
[ 392.40474025]
[ 398.31135646]
[ 236.7869]
[ 320.92008189]
[ 355.90389762]
[ 316.73117576]
[ 281.11750541]
[ 312.23595599]
[ 429.36908978]
[ 338.267]
[ 405.9204]
[ 403.12845105]
[ 356.17983968]
[ 374.6201757]
[ 270.6136]
[ 270.6136]
[ 428.87328807]
[ 405.9204]
[ 334.33198804

[ 405.9204]
[ 467.52801007]
[ 238.24213411]
[ 306.9103826]
[ 370.9283155]
[ 450.59156123]
[ 270.6136]
[ 442.81966656]
[ 304.4403]
[ 306.74106197]
[ 304.4403]
[ 405.9204]
[ 439.7471]
[ 445.03948489]
[ 304.20887449]
[ 307.00920467]
[ 338.267]
[ 439.7471]
[ 321.14488582]
[ 429.00208369]
[ 439.7471]
[ 439.7471]
[ 343.88678107]
[ 364.94194287]
[ 300.76672531]
[ 439.7471]
[ 372.0937]
[ 384.4568933]
[ 439.7471]
[ 236.7869]
[ 372.0937]
[ 453.48452655]
[ 405.9204]
[ 405.9204]
[ 253.66527198]
[ 338.267]
[ 270.6136]
[ 454.68975726]
[ 281.55143739]
[ 305.42513065]
[ 244.58033584]
[ 446.54588214]
[ 439.7471]
[ 393.38166366]
[ 304.4403]
[ 304.4403]
[ 439.7471]
[ 403.27769387]
[ 439.7471]
[ 403.59750665]
[ 396.6355981]
[ 405.9204]
[ 372.0937]
[ 434.86811152]
[ 444.58890429]
[ 338.267]
[ 439.7471]
[ 270.6136]
[ 377.57977996]
[ 405.9204]
[ 338.267]
[ 367.27403388]
[ 202.3377033]
[ 304.4403]
[ 253.03692035]
[ 304.4403]
[ 405.9204]
[ 439.7471]
[ 338.267]
[ 388.31568656]
[ 378.88364158]
[ 305.04639775]
[ 

[ 374.48643622]
[ 439.7471]
[ 390.85087502]
[ 304.4403]
[ 405.9204]
[ 411.45589915]
[ 439.7471]
[ 405.9204]
[ 304.4403]
[ 422.43914521]
[ 426.79435817]
[ 304.4403]
[ 388.55851663]
[ 304.4403]
[ 390.99301446]
[ 413.06445992]
[ 439.7471]
[ 236.7869]
[ 429.74910441]
[ 392.10684291]
[ 304.4403]
[ 405.9204]
[ 304.4403]
[ 243.71002601]
[ 346.4682594]
[ 256.63087634]
[ 386.98837981]
[ 372.0937]
[ 414.96702302]
[ 353.99861655]
[ 439.7471]
[ 372.0937]
[ 389.75654858]
[ 439.7471]
[ 427.19060815]
[ 405.9204]
[ 304.4403]
[ 278.90179445]
[ 270.6136]
[ 405.9204]
[ 439.7471]
[ 405.9204]
[ 410.28469575]
[ 420.20966595]
[ 442.31041995]
[ 405.9204]
[ 313.84369449]
[ 372.0937]
[ 304.4403]
[ 286.90153293]
[ 376.0561193]
[ 436.19364253]
[ 405.9204]
[ 394.45693505]
[ 438.78553265]
[ 439.7471]
[ 405.81111443]
[ 439.7471]
[ 236.7869]
[ 440.31845179]
[ 439.7471]
[ 409.44086713]
[ 439.7471]
[ 439.7471]
[ 276.49717714]
[ 416.03100291]
[ 415.17044557]
[ 280.73224641]
[ 405.9204]
[ 405.9204]
[ 304.4403]
[ 432.5470

[ 431.77966777]
[ 406.32393733]
[ 281.70400811]
[ 420.30114091]
[ 457.46462336]
[ 304.4403]
[ 439.7471]
[ 338.267]
[ 439.7471]
[ 304.4403]
[ 439.7471]
[ 259.28847505]
[ 405.9204]
[ 372.0937]
[ 358.37338386]
[ 337.95139272]
[ 440.40016642]
[ 405.9204]
[ 383.41057715]
[ 405.9204]
[ 435.89210978]
[ 361.34246308]
[ 387.15073714]
[ 222.51706697]
[ 304.4403]
[ 278.64980508]
[ 362.70072684]
[ 372.0937]
[ 338.68252904]
[ 379.10480035]
[ 266.57356603]
[ 338.267]
[ 304.87007651]
[ 405.9204]
[ 376.44727624]
[ 439.7471]
[ 439.7471]
[ 405.9204]
[ 359.1748632]
[ 405.9204]
[ 405.9204]
[ 313.72744731]
[ 276.47718431]
[ 304.4403]
[ 304.4403]
[ 439.7471]
[ 284.25655366]
[ 439.7471]
[ 270.6136]
[ 405.9204]
[ 270.6136]
[ 385.53370795]
[ 304.4403]
[ 220.64494537]
[ 405.9204]
[ 338.267]
[ 236.7869]
[ 337.70067556]
[ 442.77849284]
[ 405.9204]
[ 304.4403]
[ 372.0937]
[ 372.13635764]
[ 405.9204]
[ 345.64183194]
[ 446.21117297]
[ 331.66063438]
[ 277.99919511]
[ 338.267]
[ 213.81279205]
[ 384.60176766]
[ 338.267

...

[ 434.03186122]
[ 372.0937]
[ 386.7325376]
[ 439.7471]
[ 439.7471]
[ 338.267]
[ 405.9204]
[ 270.6136]
[ 371.47479377]
[ 439.7471]
[ 304.4403]
[ 304.4403]
[ 305.02052097]
[ 304.4403]
[ 405.9204]
[ 339.07907445]
[ 405.9204]
[ 405.9204]
[ 395.15263591]
[ 408.92209352]
[ 269.45546342]
[ 439.7471]
[ 281.72859002]
[ 428.20535365]
[ 236.7869]
[ 405.9204]
[ 395.88217843]
[ 415.41472352]
[ 304.4403]
[ 367.93259177]
[ 362.34810407]
[ 334.15637887]
[ 331.40953786]
[ 271.5329857]
[ 270.6136]
[ 227.04764558]
[ 405.9204]
[ 444.56704608]
[ 357.47966895]
[ 304.4403]
[ 439.7471]
[ 439.7471]
[ 252.7783644]
[ 372.0937]
[ 405.956092]
[ 304.4403]
[ 216.14687688]
[ 405.9204]
[ 338.267]
[ 307.54231242]
[ 236.7869]
[ 446.91253156]
[ 304.4403]
[ 439.7471]
[ 372.0937]
[ 348.85417686]
[ 418.13966859]
[ 439.49440089]
[ 293.05405618]
[ 338.267]
[ 270.6136]
[ 439.7471]
[ 405.9204]
[ 405.9204]
[ 441.72591296]
[ 270.06895067]
[ 236.7869]
[ 277.85464479]
[ 291.00008182]
[ 399.9211587]
[ 439.7471]
[ 446.26683131]
[ 405

[ 358.85806403]
[ 317.232566]
[ 270.6136]
[ 398.04271234]
[ 372.0937]
[ 433.17425031]
[ 448.26014966]
[ 405.9204]
[ 236.02506257]
[ 270.39099885]
[ 344.94692359]
[ 372.0937]
[ 405.9204]
[ 414.33076776]
[ 372.0937]
[ 405.9204]
[ 439.7471]
[ 428.64078015]
[ 405.9204]
[ 413.29998315]
[ 373.63208137]
[ 381.69932303]
[ 405.9204]
[ 372.0937]
[ 383.47245282]
[ 277.09866334]
[ 284.43094382]
[ 236.7869]
[ 333.76222485]
[ 435.87933818]
[ 283.99208261]
[ 236.7869]
[ 405.9204]
[ 457.26407498]
[ 440.49561396]
[ 304.4403]
[ 405.9204]
[ 439.7471]
[ 405.9204]
[ 439.7471]
[ 304.4403]
[ 236.7869]
[ 270.6136]
[ 405.9204]
[ 270.6136]
[ 405.9204]
[ 405.9204]
[ 338.267]
[ 405.9204]
[ 270.6136]
[ 331.27772863]
[ 473.5738]
[ 402.26435939]
[ 400.55757228]
[ 440.43463603]
[ 272.18574907]
[ 410.09810746]
[ 405.9204]
[ 386.01524482]
[ 439.7471]
[ 439.7471]
[ 257.63435958]
[ 338.267]
[ 453.63059872]
[ 331.25474126]
[ 422.65941537]
[ 358.00798715]
[ 316.78497295]
[ 304.4403]
[ 226.80615447]
[ 338.267]
[ 338.267]
[ 

[ 346.00058133]
[ 389.6477922]
[ 439.7471]
[ 270.6136]
[ 251.51984046]
[ 405.9204]
[ 420.4761535]
[ 453.0375661]
[ 338.267]
[ 439.7471]
[ 405.9204]
[ 447.95687762]
[ 304.4403]
[ 270.6136]
[ 314.03323535]
[ 394.42431197]
[ 405.9204]
[ 395.94306481]
[ 270.6136]
[ 439.7471]
[ 258.50953845]
[ 259.95991348]
[ 435.42557231]
[ 263.9880562]
[ 405.9204]
[ 304.4403]
[ 350.7321803]
[ 405.9204]
[ 235.71058063]
[ 405.9204]
[ 338.267]
[ 236.7869]
[ 444.73691836]
[ 304.4403]
[ 304.4403]
[ 366.90090096]
[ 384.17063821]
[ 401.07638043]
[ 304.4403]
[ 270.6136]
[ 384.0096351]
[ 291.9643078]
[ 338.267]
[ 439.7471]
[ 277.75121499]
[ 405.9204]
[ 385.59327191]
[ 248.36742614]
[ 333.39294774]
[ 236.7869]
[ 426.17547864]
[ 310.1813734]
[ 405.9204]
[ 150.46384177]
[ 439.7471]
[ 421.35094539]
[ 347.35772855]
[ 405.9204]
[ 270.6136]
[ 410.23314393]
[ 338.267]
[ 372.0937]
[ 304.4403]
[ 357.89729438]
[ 236.7869]
[ 421.60892192]
[ 304.4403]
[ 260.84355546]
[ 432.82336894]
[ 393.28481379]
[ 454.91389722]
[ 398.876381

[ 253.25327472]
[ 405.9204]
[ 269.22418596]
[ 402.54486764]
[ 257.3389294]
[ 349.61336265]
[ 405.37442315]
[ 222.24803163]
[ 270.6136]
[ 312.29423503]
[ 304.4403]
[ 379.62122855]
[ 399.18392438]
[ 338.267]
[ 394.14030713]
[ 439.7471]
[ 412.90359223]
[ 304.4403]
[ 348.27592597]
[ 448.85881701]
[ 387.54147595]
[ 438.76594249]
[ 361.26606879]
[ 405.9204]
[ 405.9204]
[ 270.6136]
[ 304.4403]
[ 196.96390365]
[ 405.32002733]
[ 405.9204]
[ 304.4403]
[ 439.7471]
[ 316.9818571]
[ 236.7869]
[ 420.33340159]
[ 383.85272155]
[ 264.59329395]
[ 439.7471]
[ 281.66116525]
[ 405.9204]
[ 440.0687646]
[ 341.69934338]
[ 372.0937]
[ 236.7869]
[ 304.4403]
[ 371.9703649]
[ 270.6136]
[ 439.7471]
[ 436.4507243]
[ 405.9204]
[ 304.4403]
[ 439.7471]
[ 277.77328281]
[ 236.7869]
[ 304.4403]
[ 304.4403]
[ 304.06501941]
[ 272.57739641]
[ 405.9204]
[ 357.87285531]
[ 380.63575492]
[ 405.9204]
[ 405.9204]
[ 370.95186744]
[ 270.6136]
[ 352.22736318]
[ 304.4403]
[ 383.52261672]
[ 421.33686322]
[ 349.6109596]
[ 270.6136]
[ 4

[ 401.89747706]
[ 338.267]
[ 270.6136]
[ 449.32172671]
[ 426.14753428]
[ 304.4403]
[ 194.68735505]
[ 383.5926908]
[ 304.4403]
[ 444.44296351]
[ 338.267]
[ 439.7471]
[ 304.4403]
[ 363.83464561]
[ 405.9204]
[ 283.29063798]
[ 405.9204]
[ 439.7471]
[ 270.6136]
[ 304.4403]
[ 405.9204]
[ 270.6136]
[ 338.267]
[ 439.7471]
[ 276.64213945]
[ 411.2485835]
[ 405.9204]
[ 439.7471]
[ 437.58195964]
[ 421.47309199]
[ 236.7869]
[ 438.24160472]
[ 315.04043835]
[ 209.21493383]
[ 217.2796063]
[ 439.7471]
[ 436.08911219]
[ 270.6136]
[ 338.267]
[ 405.9204]
[ 381.251876]
[ 405.9204]
[ 430.59351817]
[ 427.78516856]
[ 245.26651786]
[ 321.56042651]
[ 380.32423339]
[ 304.4403]
[ 337.66678197]
[ 338.267]
[ 236.7869]
[ 439.7471]
[ 244.98646758]
[ 341.99790341]
[ 270.6136]
[ 405.9204]
[ 384.52422652]
[ 309.59608249]
[ 405.9204]
[ 300.19193439]
[ 405.9204]
[ 304.4403]
[ 456.54691571]
[ 389.07054267]
[ 278.98192598]
[ 405.9204]
[ 330.79345662]
[ 397.14031136]
[ 304.4403]
[ 395.96572369]
[ 338.267]
[ 220.06741334]
[ 3

In [16]:
mae = mse**(1/2)
print(mae)
print(mae**2)

[ 362.53834018]
[ 131434.04809794]
